In [1]:
%load_ext autoreload
%autoreload 2

In [34]:
import sys
sys.path.append("..")

import numpy as np
import matplotlib.pyplot as plt
import os
from src import data, models
import json

In [54]:
from scripts.faithfulness_baselines import load_raw_results

############################################
path = "../results/faithfulness_baselines"
model_name = "gptj"
############################################


results_raw = load_raw_results(model_name=model_name, results_path=path)

In [55]:
mt = models.load_model("gptj", device="cuda")

In [63]:
relation = "plays pro sport"

relation_result = [
    r for r in results_raw if r["relation_name"] == relation
][0]

relation_result.keys()

dict_keys(['relation_name', 'total_samples', 'prompt_template', 'h_layer', 'beta', 'expected_recall', 'trials'])

In [64]:
prompt_template = relation_result['prompt_template']
h_layer = relation_result['h_layer']

trial = relation_result['trials'][0]
trial.keys()

dict_keys(['icl_prompt', 'known', 'train', 'zero_shot', 'icl'])

In [73]:
trial['icl']

{'logit_lens': {'recall': [0.24786324786324787,
   0.43162393162393164,
   0.5256410256410257,
   0.594017094017094,
   0.6324786324786325,
   0.6709401709401709,
   0.6837606837606838,
   0.717948717948718,
   0.7393162393162394,
   0.7692307692307693],
  'predictions': {'Aleksandar Đorđević': [' Jr',
    ' plays',
    ' ()',
    ' playing',
    ' (',
    "'s",
    ' played',
    'ich',
    ' /',
    ' -'],
   'Alex Karras': ["'",
    'ich',
    'ch',
    ' Jr',
    ' wears',
    ' football',
    ' and',
    'born',
    ' lives',
    'ian'],
   'Alex Rodriguez': [' Jr',
    ' sport',
    ' baseball',
    "'",
    ' playing',
    ' game',
    ' drinks',
    ' sports',
    ' career',
    ' games'],
   'Alexander Ovechkin': [' hockey',
    ' Russian',
    ' loves',
    ' plays',
    ' Jr',
    'nas',
    'ich',
    ' soccer',
    ' players',
    '�'],
   'Alexei Kovalev': [' hockey',
    'sk',
    'ich',
    ' Hockey',
    ' Russian',
    'ski',
    ' Jr',
    ' Sr',
    'sky',
    ' NHL

In [65]:
print(trial["icl_prompt"])

<|endoftext|>Dominik Hašek plays the sport of hockey
David Carney plays the sport of soccer
Dave DeBusschere plays the sport of baseball
Tommie Smith plays the sport of football
Kevin Love plays the sport of basketball
Sergei Makarov plays the sport of hockey
Nani plays the sport of soccer
Moe Berg plays the sport of baseball
{} plays the sport of


In [66]:
from src.data import RelationSample
from src import functional

relation_data = data.load_dataset().filter(relation_names=[relation])[0]

train = relation_data.set(samples=[
    RelationSample.from_dict(sample)
    for sample in trial["train"]
])

icl_prompt = functional.make_prompt(
    mt=mt,
    prompt_template=prompt_template,
    examples=train.samples,
    subject="{}",
)

test = relation_data.set(samples = list(set(relation_data.samples) - set(train.samples)))
test = functional.filter_relation_samples_based_on_provided_fewshots(
    mt = mt,
    test_relation=test,
    prompt_template=icl_prompt
)

In [72]:
from src.operators import TranslationBaseline

translation_estimator = TranslationBaseline(mt = mt, h_layer = h_layer, mode= 'zs')
translation_operator = translation_estimator(train)

In [68]:
from src.lens import logit_lens

logit_lens(mt = mt, h = translation_operator.bias)

([(' basketball', 17.109),
  (' football', 16.656),
  (' ice', 16.625),
  (' baseball', 16.188),
  (' soccer', 16.047),
  (' hockey', 15.859),
  (' tennis', 15.07),
  (' lac', 14.703),
  (' soft', 13.93),
  (' golf', 13.773)],
 {})

In [69]:
from scripts.faithfulness_baselines import evaluate

evaluate(
    operator=translation_operator,
    test_set = test,
    layer_name=models.determine_layer_paths(mt, [h_layer])[0],
)

{'recall': [0.7574468085106383,
  0.9319148936170213,
  0.9914893617021276,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0],
 'predictions': {'Aleksandar Đorđević': [' basketball',
   ' football',
   ' soccer',
   ' hockey',
   ' tennis',
   ' ice',
   ' baseball',
   ' golf',
   ' ',
   ' cricket'],
  'Alex Karras': [' basketball',
   ' football',
   ' baseball',
   ' hockey',
   ' soccer',
   ' tennis',
   ' ice',
   ' American',
   ' lac',
   ' golf'],
  'Alex Rodriguez': [' basketball',
   ' baseball',
   ' football',
   ' soccer',
   ' hockey',
   ' tennis',
   ' ice',
   ' golf',
   ' lac',
   ' cricket'],
  'Alexander Ovechkin': [' hockey',
   ' basketball',
   ' soccer',
   ' football',
   ' baseball',
   ' ice',
   ' tennis',
   ' lac',
   ' golf',
   ' American'],
  'Alexei Kovalev': [' hockey',
   ' basketball',
   ' ice',
   ' football',
   ' soccer',
   ' baseball',
   ' tennis',
   ' lac',
   ' golf',
   ' volleyball'],
  'Alexi Lalas': [' football',
   ' basketball',
  